In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
sys.path.append('..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm

import statsmodels.tsa.api as smt

import pandas as pd
import numpy as np
import scipy

In [ ]:
from scripts.settings import *
try:
    from scripts.settings_local import *
except:
    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)


exo_filter = '*'     # All 
#exo_filter = 'ES_'  # ES only
exo_filter = 'CL'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

[print(exo) for exo in storage.exo_list(exo_filter)];

In [ ]:
class Strategy_SeasDecomp_Residuals(StrategyBase):
    name = 'Strategy_SeasDecomp_Residuals'


    def __init__(self, strategy_context):
        # Initialize parent class
        super().__init__(strategy_context)
   
    def calc_entry_rules(self, pctrank_value, z_score_value, z_score_period, rules_index):

        #sd = smt.seasonal_decompose(self.data.exo.asfreq('B').ffill().dropna().iloc[:])#.plot()
        
        # Lamb values can be an optimized value
        
        #resid = self.data.exo - smt.filters.hpfilter(self.data.exo, lamb=50000)[1]
        #resid = self.data.exo - self.data.exo.ewm(10).mean()
        
        resid = pd.Series(NaN, index=self.data.exo.index)

        window = 10
        
        for i in range(window, len(self.data.exo)):
            hpfilt_slice = smt.filters.hpfilter(self.data.exo.iloc[:i], lamb=50000)[0]
            resid.iloc[i] = np.mean(hpfilt_slice[-1:])
        
        resid_pctrank = resid.rank(pct=True)
        
        resid_zscore = (resid - resid.rolling(z_score_period).mean()) / resid.rolling(z_score_period).std()
        
        signals_df = pd.DataFrame()
        signals_df['exo'] = self.data.exo
        
        if rules_index == 0:
            entry_signal = pd.Series(resid_pctrank >= pctrank_value, name='entry_signal')
            return signals_df.join(entry_signal).fillna(False).entry_signal
        
        if rules_index == 1:
            entry_signal = pd.Series(resid_pctrank <= pctrank_value, name='entry_signal')
            return signals_df.join(entry_signal).fillna(False).entry_signal
        
        if rules_index == 2:
            entry_signal = pd.Series(resid_zscore >= z_score_value, name='entry_signal')
            return signals_df.join(entry_signal).fillna(False).entry_signal
        
        if rules_index == 3:
            entry_signal = pd.Series(resid_zscore <= -z_score_value, name='entry_signal')
            return signals_df.join(entry_signal).fillna(False).entry_signal
        
        if rules_index > 3:
            raise ValueError('Rules index parameter must be in range of 0-3')

    def calc_exit_rules(self, pctrank_value, exit_rules_index):

        #sd = smt.seasonal_decompose(self.data.exo.asfreq('B').ffill().dropna().iloc[:])#.plot()
        
        #resid = self.data.exo - smt.filters.hpfilter(self.data.exo, lamb=50000)[1]
        resid = self.data.exo - self.data.exo.ewm(10).mean()
        
        signals_df = pd.DataFrame()
        signals_df['exo'] = self.data.exo
        
        resid_pctrank = resid.rank(pct=True)
        
        resid_zero_line = pd.Series(0.0, index=resid.index)
        
        signals_df = pd.DataFrame()
        signals_df['exo'] = self.data.exo
        
        if exit_rules_index == 0:
            exit_signal = pd.Series(resid_pctrank >= pctrank_value, name='exit_signal')
            return signals_df.join(exit_signal).fillna(False).exit_signal
        
        if exit_rules_index == 1:
            exit_signal = pd.Series(resid_pctrank <= pctrank_value, name='exit_signal')
            return signals_df.join(exit_signal).fillna(False).exit_signal      
        
        if exit_rules_index == 2:
            exit_signal = pd.Series((CrossDown(resid, resid_zero_line)) | (CrossUp(resid, resid_zero_line)), name='exit_signal')
            return signals_df.join(exit_signal).fillna(False).exit_signal
        
            
    def calculate(self, params=None, save_info=False):
    #
    #
    #  Params is a tripple like (50, 10, 15), where:
    #   50 - slow MA period
    #   10 - fast MA period
    #   15 - median period
    #
    #  On every iteration of swarming algorithm, parameter set will be different.
    #  For more information look inside: /notebooks/tmp/Swarming engine research.ipynb
    #

        if params is None:
            # Return default parameters
            direction, pctrank_value, z_score_value, z_score_period, rules_index, exit_rules_index = self.default_opts()
        else:
            # Unpacking optimization params
            #  in order in self.opts definition
            direction, pctrank_value, z_score_value, z_score_period, rules_index, exit_rules_index = params

        # Defining EXO price
        px = self.data.exo
                
        entry_rule = self.calc_entry_rules(pctrank_value, z_score_value, z_score_period, rules_index)

        if direction == 1:
            #exit_rule = (CrossDown(px, trailing_stop))  # Cross down for longs
            exit_rule = self.calc_exit_rules(pctrank_value, exit_rules_index)
            
        elif direction == -1:
            
            #exit_rule = (CrossUp(px, trailing_stop))  # Cross up for shorts, Cross down for longs
            exit_rule = self.calc_exit_rules(pctrank_value, exit_rules_index)
        # Swarm_member_name must be *unique* for every swarm member
        # We use params values for uniqueness
        swarm_member_name = self.get_member_name(params)

        #
        # Calculation info
        #
        calc_info = None
        if save_info:
            calc_info = {'trailing_stop': trailing_stop}

        return swarm_member_name, entry_rule, exit_rule, calc_info

## Script settings

In [ ]:
STRATEGY_CONTEXT = {
    'strategy': { 
        'class': Strategy_SeasDecomp_Residuals,
        'exo_name': 'CL_SmartEXO_Ichi_Put_Spread_150Delta_Bi',        # <---- Select and paste EXO name from cell above
        'exo_storage': storage,          
        'opt_params': [
                        #OptParam(name, default_value, min_value, max_value, step)
                        OptParamArray('Direction', [-1]),
                        #OptParam('Pct rank value', 9, 0, 1, 0.2),
                        OptParamArray('Pct rank value', [0.1,0.9,0.8,0.2,0.3,0.7]),
                        OptParam('Z Score value', 9, 3, 3, 1),
                        OptParam('Z Score period', 9, 50, 50, 50),
                        #OptParam('Rolling_period', 9, 30, 30, 10),
                        OptParamArray('Entry rules index', np.arange(2)),
                        OptParamArray('Exit rules index', np.arange(2)),
            ],
    },
    'swarm': {
        'members_count': 2,
        'ranking_class': RankerBestWithCorrel(window_size=-1, correl_threshold=0.5),
        'rebalance_time_function': SwarmRebalance.every_friday,

    },
    'costs':{
        'manager': CostsManagerEXOFixed,
        'context': {
            'costs_options': 3.0,
            'costs_futures': 3.0,
        }
    }
}

# Backtest class based strategy

In [ ]:
smgr = Swarm(STRATEGY_CONTEXT)
smgr.run_swarm()
smgr.pick()

# Saving results to swarms directory
smgr.save('./swarms/')

### WARNING! Loading swarm from file (don't run next cell if you want new swarm instance)

In [ ]:
smgr.strategy.data.exo.plot()

In [ ]:
figsize(10,10)
smgr.picked_equity.plot(label='Picked swarm equity');
smgr.raw_equity.plot(label='Average swarm equity');
legend(loc=2);

In [ ]:
smgr.raw_swarm.plot(legend=False)

In [ ]:
smgr.picked_swarm.plot()

smgr.picked_swarm.sum(1).plot(label='smgr.picked_swarm-sum')
smgr.picked_equity.plot(label='Picked swarm equity')

In [ ]:
smgr.picked_swarm.diff().mean()

# Swarm exposure

In [ ]:
smgr.picked_exposure.sum(axis=1).rolling(10).mean().plot()

# Swarm statistics

#### Non-picked swarm stats

In [ ]:
smgr.picked_stats

## Exo information

In [ ]:
smgr.strategy.exoinfo.exo_info

In [ ]:
smgr.strategy.exoinfo.data.exo.plot()

### Global filter information (obsolete)


## Costs information (per 1-exo unit)

In [ ]:
figsize(10,5)
smgr.strategy.costs.plot()

## Margin graphs

### EXO Margin (per 1 EXO unit)

In [ ]:
smgr.strategy.exoinfo.margin().plot()

# Saving results

In [ ]:
smgr.save('./swarms/')

In [ ]:
smgr.raw_swarm#.plot()